# First bar candlestick color & morning candlestick color - Analytics

### Import Library

In [10]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import math

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [11]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [12]:
data = dataset.copy()

In [13]:
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]

def morning_close(tick):
    tick=tick[(tick.index.hour == 11) & (tick.index.minute == 30)]
    if len(tick):
        return tick[0]

data['Close_9h'] = data['Close']
day_data = data.resample('D').agg({
        'Open': 'first',
        'Close': morning_close,
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h
    })
day_data.dropna(subset=['Close'], inplace=True)

In [14]:
def get_candlestick_color(open_price, close_price):
    return 'doji' if open_price == close_price else ('white' if open_price < close_price else 'black')
day_data['first_bar_color'] = day_data.apply(lambda r: get_candlestick_color(r['Open'], r['Close_9h']), axis=1)
day_data['morning_color'] = day_data.apply(lambda r: get_candlestick_color(r['Open'], r['Close']), axis=1)

In [15]:
day_data[day_data.first_bar_color == day_data.morning_color]

,Open,Close,High,Low,Close_9h,first_bar_color,morning_color
Date,,,,,,,
2018-08-14,955.5,954.3,961.8,952.1,954.9,black,black
2018-08-16,942.4,938.1,947.3,935.5,942.0,black,black
2018-08-17,952.2,954.3,954.6,945.0,953.0,white,white
2018-08-22,959.5,956.8,960.6,953.6,959.3,black,black
2018-08-24,959.5,958.9,963.0,956.4,959.1,black,black
...,...,...,...,...,...,...,...
2024-07-25,1276.9,1272.6,1278.9,1270.7,1276.5,black,black
2024-07-26,1278.7,1278.6,1285.0,1276.8,1278.3,black,black
2024-07-29,1289.9,1291.4,1294.7,1287.8,1291.2,white,white


In [16]:
len(day_data)

1473

In [17]:
day_data.groupby(day_data.index.year).count()

,Open,Close,High,Low,Close_9h,first_bar_color,morning_color
Date,,,,,,,
2018,98,98,98,98,98,98,98
2019,240,240,240,240,239,240,240
2020,248,248,248,248,246,248,248
2021,249,249,249,249,249,249,249
2022,249,249,249,249,249,249,249
2023,248,248,248,248,247,248,248
2024,141,141,141,141,141,141,141


In [18]:
day_data[(day_data.first_bar_color != day_data.morning_color) & (day_data.first_bar_color != 'doji')]

,Open,Close,High,Low,Close_9h,first_bar_color,morning_color
Date,,,,,,,
2018-08-13,943.5,947.3,954.6,942.3,943.1,black,white
2018-08-15,958.0,958.0,962.3,946.6,958.7,white,doji
2018-08-20,946.1,950.6,951.5,942.6,945.8,black,white
2018-08-21,944.7,947.4,958.5,944.0,944.5,black,white
2018-08-28,965.9,967.2,967.6,964.4,965.7,black,white
...,...,...,...,...,...,...,...
2024-07-03,1297.1,1297.7,1309.5,1294.5,1296.5,black,white
2024-07-04,1307.0,1309.0,1313.3,1303.5,1306.7,black,white
2024-07-12,1304.9,1306.5,1310.0,1298.2,1304.6,black,white
